In [699]:
import pykat
import matplotlib.pyplot as pl
import numpy as np


%matplotlib inline

In [748]:
kat_code = """
%%% FTblock laser
###########################################################################
# Laser and input optics
l L0 1 0.0 0.0 ni

s lx2 0 ni nlensa

lens ITMX_lens0 inf nlensa nlensb

###########################################################################
%%% FTend laser

%%% FTblock Xarm
###########################################################################

# Distance from beam splitter to X arm input mirror 
s lx1 2000 nlensb nITMX1a

lens ITMX_lens $SLX_f nITMX1a nITMX1b
s sITMX_th2 0 nITMX1b nITMX1

# X arm input mirror
m2 ITMXAR 0 20u 0 nITMX1 nITMXs1 
s ITMXsub 0.2 $nsilica nITMXs1 nITMXs2
m1 ITMX 0.014 $Mloss $phi_ITMX nITMXs2 nITMX2
attr ITMX Rc -1934

# X arm length
s LX $Larm nITMX2 nETMX1

# X arm end mirror
m1 ETMX 5u $Mloss $phi_ETMX nETMX1 nETMXs1
s ETMXsub 0.2 $nsilica nETMXs1 nETMXs2
m2 ETMXAR 0 500u 0 nETMXs2 nPTX 
attr ETMX Rc 2245
attr ETMX mass 40
attr ITMX mass 40

###########################################################################
%%% FTend Xarm


%%% FTblock constants
###########################################################################
const nsilica 1.44963098985906
const nTGG 1.954
const nCalcite 1.65846
const Mloss 37.5u
const Larm 3994.4692
const fM 36.397884M
const fP 54.596826M
const TLY_f 34.5k         # Thermal lens ITMY
const TLX_f 34.5k         # Thermal lens ITMX
const SLY_f inf           # Substrate lens ITMY
const SLX_f inf           # Substrate lens ITMX
const th_f 34.5k
###########################################################################
%%% FTend constants

%%% FTblock tunings
###########################################################################
const phi_SRM 90.0
const phi_PRM 0.0
const phi_ITMX 0.0
const phi_ITMY 0.0
const phi_ETMX 0.0
const phi_ETMY 0.0
const phi_BS 0
###########################################################################
%%% FTend tunings

#detectors

bp q_refl_x x q nITMX1b
bp q_in     x q ni

maxtem 4

noxaxis
yaxis re:im
"""
basekat = pykat.finesse.kat() 
basekat.parse(kat_code)

In [756]:
kat0 = basekat.deepcopy()
kat0.parse('''
cav cavXARM ITMX nITMX2 ETMX nETMX1
''')
out0 = kat0.run()

--------------------------------------------------------------
Running kat - Started at 2018-11-15 11:45:50.345465

Finished in 0.0807788 seconds


100% | ETA:  0:00:00 | Calculating                                             

In [757]:
out0['q_in']

(3300.5998789886+209.25458352935j)

In [758]:
kat1 = basekat.deepcopy()

gauss_cmd ='gauss* laser_in L0 ni ' + str(-np.real(out0['q_in'])) + ' ' + str(np.imag(out0['q_in']))

kat1.parse(gauss_cmd)

#kat1.ITMX_lens.f.value = 1/(50e-6)
#kat1.ITMX_lens0.f.value = 1/(50e-6)
out1, T = kat1.run(getTraceData=True)

--------------------------------------------------------------
Running kat - Started at 2018-11-15 11:45:50.524982


100% | ETA:  0:00:00 | Calculating                                             


Finished in 0.147604 seconds


In [759]:
out0['q_refl_x']

(1300.5998789886+209.25458352935j)

In [760]:
q = -np.conj(out1['q_refl_x'])

print(q)

(1300.5998789886+209.25458352935j)


In [761]:
### Check
pykat.BeamParam.overlap(q,out0['q_refl_x'])

1.0

In [762]:
pykat.BeamParam.overlap(-np.conj(out0['q_in']),out1['q_in'])

1.0